# Pruebo valores de espectrogramas

Configuro los espectrogramas para obtener una buena lectura de los audios después de entrenar!

In [1]:
import os
import numpy as np
import librosa
import soundfile as sf
import sounddevice as sd
import pickle
import time

os.chdir(r'C:\Users\maxia\Desktop\Facultad\Labo de acústica\Codigo')

In [2]:
"""Primero, voy a necestiar una función que me convierta los espectrogramas a audios """

class MinMaxNormaliser:
    """MinMaxNormaliser applies min max normalisation to an array."""

    def __init__(self, min_val, max_val):
        self.min = min_val
        self.max = max_val

    def normalise(self, array):
        norm_array = (array - array.min()) / (array.max() - array.min())
        norm_array = norm_array * (self.max - self.min) + self.min
        return norm_array

    def denormalise(self, norm_array, original_min, original_max):
        array = (norm_array - self.min) / (self.max - self.min)
        array = array * (original_max - original_min) + original_min
        return array


def convert_spectrograms_to_audio(spectrogram, min_max_value, hop_length, n_iter=32):
    #Instantiate the normaliser
    min_max_normaliser = MinMaxNormaliser(0, 1)

    # reshape the log spectrogram
    log_spectrogram = spectrogram[:, :]

    # apply denormalisation
    denorm_log_spec = min_max_normaliser.denormalise(log_spectrogram, 
                                                    min_max_value["min"], 
                                                    min_max_value["max"])
    # log spectrogram -> spectrogram
    spec = librosa.db_to_amplitude(denorm_log_spec)

    # apply Griffin-Lim 
    signal = librosa.griffinlim(S=spec, n_iter=n_iter, hop_length=hop_length)

    return signal

In [3]:
#Cargo los valores de min_max.
os.chdir(r'C:\Users\maxia\Desktop\Facultad\Labo de acústica\Codigo')

with open('min_max_values_ruido.pkl', "rb") as f:
    min_max_values_noise = pickle.load(f)

with open('min_max_values.pkl', "rb") as f:
    min_max_values_clean = pickle.load(f)



In [4]:
#Para acceder a la llave de cada diccionario me las tengo que rebuscar un poco porque se guardó todo el path
#en el nombre. Por ej:

keys = []
for key in min_max_values_noise.keys():
  keys.append(key)

print(keys[0])

/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo/stft_ruidoso/common_voice_es_18307009_1_SNR-0.npy.npy


In [5]:
keys = []
for key in min_max_values_clean.keys():
  keys.append(key)

print(keys[0])

/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo/stft_ruidoso/common_voice_es_18307009_1.npy.npy


In [6]:
#Por esto, para acceder a lo que hay dentro de cada llave tendría que hacer algo de este estilo:
path_key = '/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo/stft_ruidoso/'
audio_ejemplo = 'common_voice_es_18307009_2_SNR-0.npy'


min_max_values_noise[path_key + audio_ejemplo + '.npy']

{'min': -43.276955792353384, 'max': 36.723044207646616}

# Escucho los audios

In [8]:
#Primero escucho el audio limpio
sr = 16000
audio = np.load('audios_final/common_voice_es_18307009_2.npy')

clean_audio = audio/max(audio)

sd.play(clean_audio, sr)

In [9]:
#Ahora escucho la reconstrucción del espectrograma limpio
path_key = '/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo/stft_ruidoso/'
audio_ejemplo = 'common_voice_es_18307009_2.npy'

min_max = min_max_values_clean[path_key + audio_ejemplo + '.npy']
spectrogram = np.load('stft/common_voice_es_18307009_2.npy.npy')

sr = 16000
audio = convert_spectrograms_to_audio(spectrogram, min_max, hop_length=128)

clean_audio_stft = audio/max(audio)

sd.play(clean_audio_stft, sr)


In [10]:
#Ahora escucho el audio con ruido
sr = 16000
audio = np.load('audios_ruidosos/common_voice_es_18307009_2_SNR-0.npy')

noisy_audio = audio/max(audio)

sd.play(noisy_audio, sr)


In [11]:
#Ahora escucho la reconstrucción del espectrograma con ruido
path_key = '/Users/maxia/Desktop/Facultad/Labo de acústica/Codigo/stft_ruidoso/'
audio_ejemplo = 'common_voice_es_18307009_2_SNR-0.npy'

min_max = min_max_values_noise[path_key + audio_ejemplo + '.npy']
spectrogram = np.load('stft_ruidoso/common_voice_es_18307009_2_SNR-0.npy.npy')

sr = 16000
audio = convert_spectrograms_to_audio(spectrogram, min_max, hop_length=128)

noisy_audio_stft = audio/max(audio)

sd.play(noisy_audio_stft, sr)


Se puede ver que, claramente, no están bien elegidos los audios y tienen una reconstrucción mega robótica!!

Ahora voy a tratar de armar STFT pero según lo recomendado para voz en la página de librosa.

Según ellos:

* El valor n_fft=2048 samples, y una frecuencia de sampleo de 22050 Hz, corresponden a una ventana de 93 milisegundos y se considera un valor óptimo para señales musicales
* Por otro lado, para procesamiento de señales de voz, se recomienda un  n_fft=512, correspondientes a una ventana de 23 milisegundos a un samplerate de 22050 Hz. 
* Si se elige cualquier otro valor para n_fft debe ser necesariamente una potencia de dos para optimizar la velocidad del algoritmo de FFT

Mi amigo valerio recomienda:

* N_FFT = FRAME_SIZE = 512
* HOP_LENGTH = 256
* DURATION = 0.74  # in seconds
* SAMPLE_RATE = 22050


Veamos qué largo tendría que tener la señal si quiero un espectrograma de 256*256 si tengo esos valores de frame_size y esta nueva frecuencia de sampleo

In [3]:
sr = 22050
samples = 65280
data = np.zeros(samples)

#Genero el espectrograma logarítmico:
stft = librosa.stft(data,
                    n_fft=512,
                    hop_length=256)[:-1]

spectrogram = np.abs(stft)
log_spectrogram = librosa.amplitude_to_db(spectrogram)

print(f'Tengo un total de {len(data)} muestras')
print(f'Esto equivale a {len(data)/sr} segundos')
print(f'El espectrograma tiene una dimensión de: {np.shape(log_spectrogram)}')

Tengo un total de 65280 muestras
Esto equivale a 2.9605442176870747 segundos
El espectrograma tiene una dimensión de: (256, 256)


In [4]:
#Hago otra prueba
sr = 22050
samples = 65024
data = np.zeros(samples)

#Genero el espectrograma logarítmico:
stft = librosa.stft(data,
                    n_fft=1024,
                    hop_length=512)[:-1]

spectrogram = np.abs(stft)
log_spectrogram = librosa.amplitude_to_db(spectrogram)

print(f'Tengo un total de {len(data)} muestras')
print(f'Esto equivale a {len(data)/sr} segundos')
print(f'El espectrograma tiene una dimensión de: {np.shape(log_spectrogram)}')

Tengo un total de 65024 muestras
Esto equivale a 2.948934240362812 segundos
El espectrograma tiene una dimensión de: (512, 128)


In [31]:
#Con lo cual voy a necestiar un audio más largo, lo genero a partir de recortar uno que ya tengo

audio_clean_1 = np.load('audios_final/common_voice_es_18307761_1.npy')
audio_clean_2 = np.load('audios_final/common_voice_es_18307761_2.npy')
audio_clean_3 = np.load('audios_final/common_voice_es_18307761_3.npy')
audio_clean = np.array([*audio_clean_1, *audio_clean_2, *audio_clean_3]) #Concateno el audio 
audio_clean = librosa.resample(audio_clean, orig_sr=16000, target_sr=22050) #resampleo

audio_noise_1 = np.load('audios_ruidosos/common_voice_es_18307761_1_SNR-0.npy')
audio_noise_2 = np.load('audios_ruidosos/common_voice_es_18307761_2_SNR-0.npy')
audio_noise_3 = np.load('audios_ruidosos/common_voice_es_18307761_3_SNR-0.npy')
audio_noise = np.array([*audio_noise_1, *audio_noise_2, *audio_noise_3]) #Concateno el audio 
audio_noise = librosa.resample(audio_noise, orig_sr=16000, target_sr=22050) #resampleo

In [34]:
#STFT audio limpio:
audio_clean_test_1 = audio_clean[0:65280]
stft_clean = librosa.stft(audio_clean_test_1, n_fft=512, hop_length=256)[:-1]
print(np.shape(stft_clean))
# Lo paso a valor absoluto y pierdo la fase
spectrogram = np.abs(stft_clean)
#Lo paso a logarítmico
log_spectrogram = librosa.amplitude_to_db(spectrogram)
print(np.shape(log_spectrogram))
# Lo vuelvo a amplitud porque soy re loco y no me importa nada
spec = librosa.db_to_amplitude(log_spectrogram)
print(np.shape(spec))
# Aplico Griffin-Lim y recupero la señal
recovered_signal_1 = librosa.griffinlim(spec, hop_length=256, n_iter=64)

sd.play(audio_clean_test_1, 22050)
time.sleep(len(audio_clean_test_1)/22050)
sd.play(recovered_signal_1, 22050)



(256, 256)
(256, 256)
(256, 256)


In [36]:
#STFT audio limpio:
audio_clean_test_2 = audio_clean[0:65024]
stft_clean = librosa.stft(audio_clean_test_2, n_fft=1024, hop_length=512)[:-1]
print(np.shape(stft_clean))
# Lo paso a valor absoluto y pierdo la fase
spectrogram = np.abs(stft_clean)
#Lo paso a logarítmico
log_spectrogram = librosa.amplitude_to_db(spectrogram)
print(np.shape(log_spectrogram))
# Lo vuelvo a amplitud porque soy re loco y no me importa nada
spec = librosa.db_to_amplitude(log_spectrogram)
print(np.shape(spec))
# Aplico Griffin-Lim y recupero la señal
recovered_signal_2 = librosa.griffinlim(spec, hop_length=512)

sd.play(audio_clean_test_2, 22050)
time.sleep(len(audio_clean_test_2)/22050)
sd.play(recovered_signal_2, 22050)

(512, 128)
(512, 128)
(512, 128)


In [37]:
#STFT audio limpio:
audio_clean_test_3 = audio_clean[0:65024]
stft_clean = librosa.stft(audio_clean_test_3, n_fft=256, hop_length=128)[:-1]
print(np.shape(stft_clean))
# Lo paso a valor absoluto y pierdo la fase
spectrogram = np.abs(stft_clean)
#Lo paso a logarítmico
log_spectrogram = librosa.amplitude_to_db(spectrogram)
print(np.shape(log_spectrogram))
# Lo vuelvo a amplitud porque soy re loco y no me importa nada
spec = librosa.db_to_amplitude(log_spectrogram)
print(np.shape(spec))
# Aplico Griffin-Lim y recupero la señal
recovered_signal_3 = librosa.griffinlim(spec, hop_length=128)

sd.play(audio_clean_test_3, 22050)
time.sleep(len(audio_clean_test_2)/22050)
sd.play(recovered_signal_3, 22050)

(128, 509)
(128, 509)
(128, 509)


In [162]:
sf.write('original.wav', audio_clean_test_1, 22050)
sf.write('recovered_512.wav', recovered_signal_1, 22050)
sf.write('recovered_1024.wav', recovered_signal_2, 22050)
sf.write('recovered_256.wav', recovered_signal_3, 22050)